In [ ]:
%load_ext autoreload
%autoreload 2

from butao.env import TaoEnv, get_root_config
from butao.data import TaoData
from butao.model import TaoModel
from butao.infer import TaoInfer

## choose config

In [ ]:
config = "detectnet_v2"  # maps to butao/configs/detectnet_v2.yaml

## get configs, make dirs and mount file

In [ ]:
config_path = get_root_config(config=config)
env = TaoEnv(config_path)
data = TaoData(config_path)
model = TaoModel(config_path)
infer = TaoInfer(config_path)

env.make_mount_file()

*Note: Please make sure to remove any stray artifacts/files from the `env.user_experiment_dir` or `env.data_download_dir` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

In [34]:
print(f"local workspace: \n{env.local_project_dir}\n")
print(f"data and model spec files must be copied to: \n{env.local_specs_dir}\n")
print(f"dataset will be in: \n{env.local_data_dir}\n")
print(f"TAO experiment outputs will be in: \n{env.local_experiment_dir}\n")

local workspace: 
/home/david/butao/notebooks/detectnet_v2

data and model spec files must be copied to: 
/home/david/butao/notebooks/detectnet_v2/specs

dataset will be in: 
/home/david/butao/notebooks/detectnet_v2/data

TAO experiment outputs will be in: 
/home/david/butao/notebooks/detectnet_v2/detectnet_v2



## downloads data and converts to tfrecords

In [ ]:
data.download()

In [ ]:
data.print_downloaded_data_info()

In [ ]:
!tao {env.tao_model_name} dataset_convert -d {data.spec_file} -o {data.tfrecords_dir}

## downloads model

In [ ]:
!ngc registry model list {model.ngc_registry}:*

In [ ]:
!ngc registry model download-version {model.ngc_registry}:{model.model_version} \
    --dest {model.local_model_dir}

## trains

In [ ]:
model.show_hyperparams(hparams=[
    "min_learning_rate", "max_learning_rate", "batch_size_per_gpu"
])

*Note: current learning rates assume single-gpu training. It's a good idea to adjust them for multi-gpu training, since adding gpus increases total batch size*

In [ ]:
model.adjust_train_spec(hparams={
    "min_learning_rate": "5e-05",
    "max_learning_rate": "5e-03",
    "batch_size_per_gpu": 8
})

In [ ]:
!tao detectnet_v2 train -e {model.train_spec} \
                        -r {model.user_experiment_dir_unpruned} \
                        -k {model.key} \
                        -n {model.model_version} \
                        --gpus {model.num_gpus}

## Visualize inferences
In this section, we run the inference tool to generate inferences on the trained models. To render bboxes from more classes, please edit the inference spec file to include all the classes you would like to visualize and edit the rest of the file accordingly.

In [ ]:
print(f"Copy some test images here: {infer.local_sample_data_dir}")
print(f"Edit this spec file here: {infer.local_inference_spec}")

In [ ]:
# Running inference for detection on n images
!tao detectnet_v2 inference -e {infer.inference_spec} \
                            -o {infer.infer_dir} \
                            -i {infer.sample_data_dir} \
                            -k {model.key}

In [ ]:
print(f"""
    The inference tool produces two outputs.
    
    1. Overlain images in:
    {infer.local_annotated_dir}
    
    2. Frame by frame bbox labels in kitti format located in:
    {infer.local_predictions_dir}
""")

*Note: To run inferences for a single image, simply replace the path to the -i flag in inference command with the **docker image's path** to the image.*

In [ ]:
infer.plot_image_grid(rows=3, cols=2, num_images=5)